In [39]:
from imports import *
from funcs import *

import funcs as myfuncs
import funcs as myimps

import importlib

importlib.reload(myfuncs);
importlib.reload(myimps);

In [50]:
climate_var = "climate-data\A2\HADCM3_A2b_TEMP_2050.dif"

global temps_mat

temps_mat = conv(climate_var)[0]
temps_df = conv(climate_var)[1]

month_dict = dict(zip(month, range(0,12)))

fig = plt.figure()
plt.rcParams['figure.figsize'] = [10, 20]

def plotter(month):
    ax = plt.axes(projection=ccrs.PlateCarree())

    data, lons = add_cyclic_point(temps_mat[month_dict[month]], coord = longitudes)

    maxv = np.max(temps_mat)
    minv = np.min(temps_mat)
    dif = maxv - minv

    if 'WIND' in climate_var:
        cs = ax.contourf(lons, latwind, data, levels = np.arange(minv, maxv, dif/10),
            transform = ccrs.PlateCarree(),cmap='Blues',extend='both')
        
        cb = plt.colorbar(cs, ticks = cs.levels, orientation = 'horizontal', location = 'top', 
                shrink = 0.75, aspect = 50, pad = 0.01)
        cb.set_label('Scalar Wind Speed Gradient [m/g]', fontsize = 8)
        cb.ax.tick_params(labelsize=8)
    
    else:
        cs = ax.contourf(lons, latitudes, data, levels = np.arange(minv, maxv, dif/10),
            transform = ccrs.PlateCarree(),cmap='Reds',extend='both')
        
        cb = plt.colorbar(cs, ticks = cs.levels, orientation = 'horizontal', location = 'top', 
                shrink = 0.75, aspect = 50, pad = 0.01)
        cb.set_label('Daily Mean Temperature Difference [°C]', fontsize = 8)
        cb.ax.tick_params(labelsize=8)
        



    ax.coastlines()

    ax.set_xticks(np.arange(-180,181,30), crs=ccrs.PlateCarree())
    lon_formatter = cticker.LongitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)

    ax.set_yticks(np.arange(-90,91,15), crs=ccrs.PlateCarree())    
    lat_formatter = cticker.LatitudeFormatter()
    ax.yaxis.set_major_formatter(lat_formatter)


    ax.tick_params(axis='both', which='major', labelsize=8)

    ax.gridlines()
    plt.rcParams['figure.figsize'] = [10, 20]

    return 

month_slider = widgets.SelectionSlider(
    options=month,
    value='Jan',
    description='Month',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

'''month_slider = widgets.IntSlider(
                value=0,
                min=0,
                max=11,
                step=1,
                continuous_update=True,
                readout=True,
                description = 'Month',)'''
    
interact(plotter, month = month_slider)

#
plt.show()


<Figure size 1000x2000 with 0 Axes>

interactive(children=(SelectionSlider(continuous_update=False, description='Month', options=('Jan', 'Feb', 'Ma…